# Airport Page Rank

#### Load Data

In [0]:
# load packages
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from datetime import datetime

from pyspark.sql import functions as f
from pyspark.sql.functions import col, to_timestamp, to_utc_timestamp, concat_ws, udf
from pyspark.sql import Window

from graphframes import *

sc = spark.sparkContext

# set blob paths
blob_container = "w261-team28-container" 
storage_account = "team28" 
secret_scope = "w261-team28-scope"
secret_key = "w261-team28-key"
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"

spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

# set data retrieval path
ROOT_PATH = 'dbfs:/mnt/mids-w261/datasets_final_project/'
ROOT_TEAM28_PATH = blob_url

# airline path
PARQ_AIRLINE_PATH = ROOT_PATH + 'parquet_airlines_data/'

# raw data path
AIRLINE_FULL_PATH = PARQ_AIRLINE_PATH + '201*.parquet'
AIRLINE_3M_PATH = ROOT_PATH + 'parquet_airlines_data_3m/*.parquet'
AIRLINE_6M_PATH = ROOT_PATH + 'parquet_airlines_data_6m/*.parquet'

# processed data path
AIRLINE_FULL_PROCESSED_PATH = blob_url + '/full_airline_data-processed'
AIRLINE_3M_PROCESSED_PATH = blob_url + '/processed/airline_3m_data.parquet'
AIRLINE_2015_PROCESSED_PATH = blob_url + '/processed/airline_2015_data.parquet'
AIRLINE_2019_PROCESSED_PATH = blob_url + '/airline_2019_data-processed'

WEATHER_2015_PROCESSED_PATH = blob_url + '/processed/weather_2015_data.parquet'

JOINED_3M_PROCESSED_PATH = blob_url + '/processed/joined_3m_data.parquet'
JOINED_2015_PROCESSED_PATH = blob_url + '/processed/joined_2015_data.parquet'

# read raw data
airline_full_raw_df = spark.read.parquet(AIRLINE_FULL_PATH).cache()
airline_3m_raw_df = spark.read.parquet(AIRLINE_3M_PATH).cache()
airline_6m_raw_df = spark.read.parquet(AIRLINE_6M_PATH).cache()

# read processed data
airline_full_processed_df = spark.read.parquet(AIRLINE_FULL_PROCESSED_PATH).cache()
airline_2015_processed_df = spark.read.parquet(AIRLINE_2015_PROCESSED_PATH).cache()
airline_2019_processed_df = spark.read.parquet(AIRLINE_2019_PROCESSED_PATH).cache()

joined_3m_processed_df = spark.read.parquet(JOINED_3M_PROCESSED_PATH).cache()
joined_2015_processed_df = spark.read.parquet(JOINED_2015_PROCESSED_PATH).cache()

# set main_data
main_data = airline_full_processed_df

#### Investigate Blob Azure Storage

In [0]:
display(dbutils.fs.ls(blob_url))

path,name,size
wasbs://w261-team28-container@team28.blob.core.windows.net/30k_weather_sample_processed/,30k_weather_sample_processed/,0
wasbs://w261-team28-container@team28.blob.core.windows.net/3m_flights_processed/,3m_flights_processed/,0
wasbs://w261-team28-container@team28.blob.core.windows.net/6m_flights_processed/,6m_flights_processed/,0
wasbs://w261-team28-container@team28.blob.core.windows.net/ICAO_station_mapping/,ICAO_station_mapping/,0
wasbs://w261-team28-container@team28.blob.core.windows.net/airline_2019_data-processed/,airline_2019_data-processed/,0
wasbs://w261-team28-container@team28.blob.core.windows.net/data_for_aditya/,data_for_aditya/,0
wasbs://w261-team28-container@team28.blob.core.windows.net/full_airline_data-processed/,full_airline_data-processed/,0
wasbs://w261-team28-container@team28.blob.core.windows.net/graph_test/,graph_test/,0
wasbs://w261-team28-container@team28.blob.core.windows.net/processed/,processed/,0
wasbs://w261-team28-container@team28.blob.core.windows.net/weather_2015_02_21/,weather_2015_02_21/,0


#### Data Size

In [0]:
def file_ls(path: str):
    '''List all files in base path recursively.'''
    tot = 0
    for x in dbutils.fs.ls(path):
        if x.path[-1] != '/':
            tot += x.size
            yield x
        else:
            for y in file_ls(x.path):
                yield y
    yield f'DATASIZE: {tot}'

total_size = []
for i in file_ls(AIRLINE_2019_PROCESSED_PATH):
    if 'DATASIZE:' in i:
        total_size.append(int(i.split(' ')[1]))

print(f'Total Data Size: {sum(total_size)/1e9:.2f} GB')
# print(f'Total Number of Records: {main_data.count():,}')

Total Data Size: 0.44 GB

In [0]:
main_data.printSchema()

root
-- YEAR: integer (nullable = true)
-- QUARTER: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: string (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- OP_CARRIER_FL_NUM: integer (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: integer (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CANCELLED: double (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)
-- ID: string (nullable = true)
-- IATA_ORIGIN: string (nullable = true)
-- ICAO_ORIGIN: string (nullable = true)
-- AIRPORT_LAT_ORIGIN: double (nullable = true)
-- AIRPORT_LONG_ORIGIN: double (nullable = true)
-- AIRPORT_TIMEZONE_ORIGIN: string (nullable = true)
-- AIRPORT_UTC_OFFSET_ORIGIN: string (nullable = true)
-- IATA_DEST: string (nullable = true)
-- ICAO_DEST: string (nullable = true)
-- AIRPORT_LAT_DEST: double (nullable = true)
-- AIRPORT_LONG_DEST: double (nullable = true)
-- AIRPORT_TIMEZONE_DEST: string (nullable = true)
-- AIRPORT_UTC_OFFSET_DEST: string (nullable = true)
-- CRS_DEP_TIME_UTC: timestamp (nullable = true)
-- DEP_TIME_UTC: timestamp (nullable = true)
-- ARR_TIME_UTC: timestamp (nullable = true)
-- CRS_ARR_TIME_UTC: timestamp (nullable = true)
-- CRS_DEP_TIME_UTC_HOUR: timestamp (nullable = true)
-- CRS_ARR_TIME_UTC_HOUR: timestamp (nullable = true)

#### Create GraphFrame + PageRank

In [0]:
ORIGIN = main_data.select('ORIGIN', 'ORIGIN_CITY_NAME', 'AIRPORT_LAT_ORIGIN', 'AIRPORT_LONG_ORIGIN').distinct()
DEST = main_data.select('DEST', 'DEST_CITY_NAME', 'AIRPORT_LAT_DEST', 'AIRPORT_LONG_DEST').distinct()

AIRPORT = ORIGIN.union(DEST).distinct()
AIRPORT = AIRPORT.withColumnRenamed('ORIGIN', 'id')\
                 .withColumnRenamed('ORIGIN_CITY_NAME', 'name')

AIRPORT_EDGES = (
    main_data.select(
        f.col('ORIGIN').alias('src'),
        f.col('DEST').alias('dst'),
        'OP_UNIQUE_CARRIER','MONTH','QUARTER','YEAR','ORIGIN_CITY_NAME','DEST_CITY_NAME','DISTANCE',
        'AIRPORT_LAT_ORIGIN', 'AIRPORT_LONG_ORIGIN', 'AIRPORT_LAT_DEST', 'AIRPORT_LONG_DEST',
        f.format_string('%d-%02d',f.col('YEAR'),f.col('MONTH')).alias('YEAR-MONTH')        
    )
).cache()

airport_graph = GraphFrame(AIRPORT, AIRPORT_EDGES)
airport_rank = airport_graph.pageRank(resetProbability=0.15, maxIter=5).cache()

In [0]:
# top 20 busiest airport hubs
airport_rank_pd_df = airport_rank.vertices.orderBy("pagerank", ascending=False).toPandas()
(airport_rank.vertices.orderBy("pagerank", ascending=False)).limit(20).toPandas()

Out[69]:

,id,name,AIRPORT_LAT_ORIGIN,AIRPORT_LONG_ORIGIN,pagerank
0,ATL,"Atlanta, GA",33.636700,-84.428101,20.274831
1,ORD,"Chicago, IL",41.978600,-87.904800,16.933371
2,DFW,"Dallas/Fort Worth, TX",32.896801,-97.038002,14.591163
3,DEN,"Denver, CO",39.861698,-104.672997,13.296385
4,LAX,"Los Angeles, CA",33.942501,-118.407997,9.765168
5,MSP,"Minneapolis, MN",44.882000,-93.221802,8.601058
6,SFO,"San Francisco, CA",37.618999,-122.375000,8.293052
7,CLT,"Charlotte, NC",35.214001,-80.943100,8.268248
8,IAH,"Houston, TX",29.984400,-95.341400,8.169985
9,DTW,"Detroit, MI",42.212399,-83.353401,7.872102


In [0]:
# normalize page rank
airport_rank_pd_df['norm_pageRank'] = airport_rank_pd_df['pagerank']/airport_rank_pd_df['pagerank'].max()
airport_rank_pd_df['norm_pageRank'] = airport_rank_pd_df['norm_pageRank'].round(2)

# label
airport_rank_pd_df['label'] = 'IATA: ' + airport_rank_pd_df['id'] + ', PageRank: ' + airport_rank_pd_df['norm_pageRank'].astype('str')

# plot geomap
fig = go.Figure(
    data=go.Scattergeo(
        
        locationmode = 'USA-states',
        lat = airport_rank_pd_df['AIRPORT_LAT_ORIGIN'],
        lon = airport_rank_pd_df['AIRPORT_LONG_ORIGIN'],
        text = airport_rank_pd_df['label'],
        mode = 'markers',
        marker = dict(size = airport_rank_pd_df['pagerank']*2,
                      color = airport_rank_pd_df['pagerank'],
                      colorbar_title = 'Rank')
    )
)

fig.update_layout(
        title = 'Y2015-2019 Airport PageRank',
        geo = dict(projection_type ='albers usa'),
    )
fig.show()